In [9]:
import torch
from torchvision.datasets import CIFAR10
from torchvision import transforms
from torch import nn 
train_data = CIFAR10(root='data', train=True, download=True, transform=transforms.ToTensor())

Files already downloaded and verified


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [13]:
class Cifar10_clf(nn.Module):
    def __init__(self):
        super(Cifar10_clf, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(2304, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
        
    def forward(self ,x):
        x = self.conv(x)
        x = self.relu(x)
        x = self.conv2d(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x
model = Cifar10_clf()

In [15]:
from torch import optim

loss_fn = nn.CrossEntropyLoss()
optim_fn = optim.AdamW(model.parameters(), 0.001)

In [ ]:
def training_loop(epochs):
    for epoch in epochs